In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt

### Instructions

Your case study is to build a classifier using logistic regression to predict hospital readmittance. 
There is missing data that must be imputed. Once again, discuss variable importances as part of your submission.

### Team:
- Dylan Scott
- Jobin Joseph
- Taylor Bonar
- Satvik Ajmera

In [4]:
#data
diabetes_df = pd.read_csv("../dataset_diabetes/diabetic_data.csv", header=0, na_values="?", low_memory=False)
diabetes_df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
diabetes_df['race'].value_counts()

Caucasian          76099
AfricanAmerican    19210
Hispanic            2037
Other               1506
Asian                641
Name: race, dtype: int64

In [8]:
diabetes_df['age'].value_counts()

[70-80)     26068
[60-70)     22483
[50-60)     17256
[80-90)     17197
[40-50)      9685
[30-40)      3775
[90-100)     2793
[20-30)      1657
[10-20)       691
[0-10)        161
Name: age, dtype: int64

In [13]:
age_values = list(diabetes_df['age'].values)
age_values

['[0-10)',
 '[10-20)',
 '[20-30)',
 '[30-40)',
 '[40-50)',
 '[50-60)',
 '[60-70)',
 '[70-80)',
 '[80-90)',
 '[90-100)',
 '[40-50)',
 '[60-70)',
 '[40-50)',
 '[80-90)',
 '[60-70)',
 '[60-70)',
 '[50-60)',
 '[50-60)',
 '[70-80)',
 '[70-80)',
 '[50-60)',
 '[60-70)',
 '[70-80)',
 '[80-90)',
 '[70-80)',
 '[50-60)',
 '[80-90)',
 '[50-60)',
 '[20-30)',
 '[80-90)',
 '[60-70)',
 '[70-80)',
 '[70-80)',
 '[60-70)',
 '[70-80)',
 '[60-70)',
 '[70-80)',
 '[60-70)',
 '[70-80)',
 '[50-60)',
 '[70-80)',
 '[40-50)',
 '[70-80)',
 '[50-60)',
 '[80-90)',
 '[40-50)',
 '[70-80)',
 '[70-80)',
 '[50-60)',
 '[60-70)',
 '[50-60)',
 '[70-80)',
 '[40-50)',
 '[50-60)',
 '[60-70)',
 '[60-70)',
 '[50-60)',
 '[40-50)',
 '[80-90)',
 '[70-80)',
 '[70-80)',
 '[50-60)',
 '[40-50)',
 '[80-90)',
 '[50-60)',
 '[90-100)',
 '[10-20)',
 '[80-90)',
 '[50-60)',
 '[50-60)',
 '[70-80)',
 '[50-60)',
 '[60-70)',
 '[70-80)',
 '[70-80)',
 '[70-80)',
 '[60-70)',
 '[60-70)',
 '[50-60)',
 '[50-60)',
 '[70-80)',
 '[50-60)',
 '[50-60)',
 '[

In [12]:
#things to do. Change male and female to 1 and 0
#change age to number
#change race to number
#look at missing values

#value dictionaries
race = {'Caucasian': 0, 'AfricanAmerican':1, 'Hispanic':2,'Other':3,'Asian':4}
gender = {'male': 1,'female': 0}
age = {}
count = 0
for i in range(len(age_values)):
    age[age_values[i]] = count
    count+=1
print(age)



{'[0-10)': 100787, '[10-20)': 101719, '[20-30)': 101688, '[30-40)': 101728, '[40-50)': 101755, '[50-60)': 101745, '[60-70)': 101760, '[70-80)': 101765, '[80-90)': 101764, '[90-100)': 101741}
